In [ ]:
# Mount the Drive to access Data

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # Drive Access

%cd /content/drive/MyDrive/Evida_NLP_Project/

/content/drive/MyDrive/Evida_NLP_Project


In [ ]:
pip install ktrain

In [ ]:
# Libraries 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0";

In [ ]:
# Libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data 

'''
In this cell we have to load the data what we are going to use.
- data : the 12 colon cancer documents and their summaries
- fifity_documents : the fifity documents no colon cancer.
In the  first experimentation, we have to use distilBErt based pretrained model. 
'''

# load the data from the excel file and choose the sheet 'ColonCancer_documents'.
data = pd.read_excel('document_data.xlsx','ColonCancer_documents')

fifity_document = pd.read_excel('fifity_document.xlsx')

In [ ]:
'''
Because of a few nomber of data for the colon cancer documents, 
we have to select 12 documents no colon cancer for this task 
in total we would have 24 documents both colon cancer and no colon cancer. 
'''
data_sel = fifity_document.iloc[0:12,:]  # We select only 12 no colon cancer documents among 50 for our task
data_selected = data_sel[['text','label','label_name']] # select the orginal text in each document , the label and label name for the no colon cancer documents

data_selected_summary = data_sel[['summary','label','label_name']] # select the orginal text in each document , the label and label name for the no colon cancer documents
data_selected['label']= 1 # Change the label value for the no colon cancer documents 
data_selected_summary['label']= 1 # Change the label value for the no colon cancer documents 
data_selected['label_name']= 'no_colonCancer' # change the labal name for the no colon cancer documents
data_selected_summary['label_name']= 'no_colonCancer' # change the labal name for the no colon cancer documents
colon_cancer_doc = data[['text','label','label_name']] # Select the orginal text in each document , the label and label name for  the colon cancer documents
colon_cancer_doc_summary = data[['summary','label','label_name']] # Select the orginal text in each document , the label and label name for  the colon cancer documents

In [ ]:
# check the shape of the data set.
print(f'fifity_document shape: {data_selected.shape}')
print(f'colon_cancer_doc shape: {colon_cancer_doc.shape}')
# check the shape of the summaries data set.
print(f'fifity_summaries_document shape: {data_selected_summary.shape}')
print(f'colon_cancer_summaries_doc shape: {colon_cancer_doc_summary.shape}')

fifity_document shape: (12, 3)
colon_cancer_doc shape: (12, 3)
fifity_summaries_document shape: (12, 3)
colon_cancer_summaries_doc shape: (12, 3)


In [ ]:
# Concatenate the colon cancer dataFrame et no colon cancer dataFrame 
all_data = pd.concat([data_selected,colon_cancer_doc])
all_data_summary = pd.concat([data_selected_summary,colon_cancer_doc_summary])
print(f'All_data  shape: {all_data.shape}') 
print(f'all_data_summary  shape: {all_data_summary.shape}') 

All_data  shape: (24, 3)
all_data_summary  shape: (24, 3)


In [ ]:
import ktrain
from ktrain import text
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Shuffle the data 
all_data = shuffle(all_data)
all_data_summary = shuffle(all_data_summary)
np.random.seed(25)

# Retrieve text content , label and associated label_name
contents,contents_summary = all_data['text'],all_data_summary['summary']
labels,label_summary = all_data['label'],all_data_summary['label']
label_names ,label_names_summary = all_data['label_name'],all_data_summary['label_name']

In [ ]:
'''
The text that the model processes must first be tokenized and then data embedded. 
To begin with, we need to transform each text into a list as a type of data structure.
We will transform the data that are in dataFrame format to list format in this cell
'''
dataset  = []       # List of the text contents of all documents
datasetlabel = []   # List of the labels 
label_name = []     # List of label names
for i in all_data['text']:
  dataset.append(i)
for j in all_data['label']:
  datasetlabel.append(j)
for t in all_data['label_name']:
  label_name.append(t)

In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(dataset, datasetlabel, test_size=0.25)

In [ ]:
print(f'train data size {len(train_texts)}')
print(f'test data size {len(test_texts)}')

train data size 18
test data size 6


In [ ]:
pd.DataFrame(test_texts).to_excel("val_dataset.xlsx")
pd.DataFrame(test_labels).to_excel("val_labels.xlsx")

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
#only for fine tunning

import torch

class colonCancerData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = colonCancerData(train_encodings, train_labels)
test_dataset = colonCancerData(test_encodings, test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_colonCancer',   # output directory
    num_train_epochs=100,                 # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    logging_dir='./logs',                 # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
trainer = Trainer(
    model=model,                            # the instantiated  transformers model to be trained
    args=training_args,                     # training arguments, defined above
    train_dataset=train_dataset,            # training dataset
    #eval_dataset=val_dataset               # evaluation dataset
)
trainer.train()

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 18
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 200


{'loss': 0.7026, 'learning_rate': 1.0000000000000002e-06, 'epoch': 5.0}
{'loss': 0.6898, 'learning_rate': 2.0000000000000003e-06, 'epoch': 10.0}
{'loss': 0.6829, 'learning_rate': 3e-06, 'epoch': 15.0}
{'loss': 0.6664, 'learning_rate': 4.000000000000001e-06, 'epoch': 20.0}
{'loss': 0.6202, 'learning_rate': 5e-06, 'epoch': 25.0}
{'loss': 0.5724, 'learning_rate': 6e-06, 'epoch': 30.0}
{'loss': 0.4697, 'learning_rate': 7.000000000000001e-06, 'epoch': 35.0}
{'loss': 0.3071, 'learning_rate': 8.000000000000001e-06, 'epoch': 40.0}
{'loss': 0.1817, 'learning_rate': 9e-06, 'epoch': 45.0}
{'loss': 0.0985, 'learning_rate': 1e-05, 'epoch': 50.0}
{'loss': 0.0549, 'learning_rate': 1.1000000000000001e-05, 'epoch': 55.0}
{'loss': 0.0287, 'learning_rate': 1.2e-05, 'epoch': 60.0}
{'loss': 0.017, 'learning_rate': 1.3000000000000001e-05, 'epoch': 65.0}
{'loss': 0.0126, 'learning_rate': 1.4000000000000001e-05, 'epoch': 70.0}
{'loss': 0.0091, 'learning_rate': 1.5e-05, 'epoch': 75.0}
{'loss': 0.0072, 'learnin



Training completed. Do not forget to share your model on huggingface.co/models =)




{'loss': 0.0037, 'learning_rate': 2e-05, 'epoch': 100.0}
{'train_runtime': 178.4054, 'train_samples_per_second': 10.089, 'train_steps_per_second': 1.121, 'train_loss': 0.2570064843446016, 'epoch': 100.0}


TrainOutput(global_step=200, training_loss=0.2570064843446016, metrics={'train_runtime': 178.4054, 'train_samples_per_second': 10.089, 'train_steps_per_second': 1.121, 'train_loss': 0.2570064843446016, 'epoch': 100.0})

In [ ]:
import torch.nn.functional as F
# please comment the next line if you want to test the fine tunned model
#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
def evalua(string):
    pt_batch = tokenizer(
    [string],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
    )

    #model_custom = pipeline('sentiment-analysis',model,None,tokenizer)
    pt_outputs = model.cpu()(**pt_batch)
    pt_predictions = F.softmax(pt_outputs.logits, dim=-1)
    
    return np.argmax(pt_predictions.detach().numpy()[0])

In [ ]:

test_texts = pd.read_excel('val_dataset.xlsx')
test_labels = pd.read_excel('val_labels.xlsx')
test_labels = test_labels[0]
labeles = []
print(test_texts)
for i in test_texts[0]:
    results = evalua(i)
    labeles.append(results)

   Unnamed: 0                                                  0
0           0  This is a patient of years, who had a personal...
1           1  We report a case of leiomyosarcoma of the colo...
2           2  This is a 72-year-old male patient with a hist...
3           3  A 46-year-old female patient hysterectomized f...
4           4  A 65-year-old man consulted for an infravesica...
5           5  A 36-year-old male, with no history of interes...


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,labeles))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      0.33      0.50         3

    accuracy                           0.67         6
   macro avg       0.80      0.67      0.62         6
weighted avg       0.80      0.67      0.62         6

